In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests 
import time
import pandas as pd 
import numpy as np

In [2]:
def property_crawler(base_url=None, start=1, end=1,snooze=1):
    '''
    
    This function loops over a specified number of pages of the website page to be scraped.
    
    The function accepts the base URL of the website,page number to start scraping from, 
    page number to end scraping at, and the number of seconds to wait in-between scraping.
    
    The function returns nothing. This is more of a progress display for the scraping process.
    
    '''    
    
    #  empty list to stare dataframes to be concatenated
    scraped_df_list = []
    
    for page_num in tqdm(range(start, end+1), desc='Scraping pages'):
        
        #  appending specific page number to the base url
        specific_url = base_url + str(page_num)
        
        #  retrieving the html of the specific url
        specific_html = requests.get(specific_url)
        
        #  status code of page: to determine if the page is accessible
        status_code = specific_html.status_code
        
        #  indicating that the process will start 
        if status_code != 200:
            print(f'Retrieval of {specific_url} failed with status code {status_code}')
            break
        print(f'Retrieving: {specific_url} --- Success!')
        
        #  function that retrieves data from the specific html and appends it to empty lists
        scraped_df_list.append(get_propertyinfo(specific_html))
        
        #  giving the spider a little break
        time.sleep(snooze)
    
    return concat_dataframelist(scraped_df_list)

In [3]:
def get_propertyinfo(specific_html=None) -> pd.DataFrame: 
    '''
    
    This function scrapes the lamudi.com.ph website for properties.
    
    The function accepts the retrieved html of a page.
    
    The function returns a dataframe containing the scraped data.
    
    '''
    
    #  empty lists to store data retrieved from page
    listing_list = []
    location_list = []
    price_list = []
    bedroom_num_list = []
    bathroom_num_list = []
    floor_area_list = [] 
    land_size_list = []
    longitude_list = []
    latitude_list = []
    
    #  list of list to pass to dataframe_fromlists()
    features_list = [listing_list,
                     location_list,
                     price_list,
                     bedroom_num_list,
                     bathroom_num_list,
                     floor_area_list,
                     land_size_list,
                     longitude_list,
                     latitude_list]
    
    #  soupify the html passed
    proplist_soup = BeautifulSoup(specific_html.text, 'lxml')
    
    #  data of each proprety is concentrated in this html div tag with the class: ListingCell-AllInfo ListingUnit
    propinfo_group = proplist_soup('div', class_ ='ListingCell-AllInfo ListingUnit')
    
    #  looping through each property listing from propinfo_group
    for property in propinfo_group:
        
        #  retrieve listing name 
        listing = property.find('h2', class_ ='ListingCell-KeyInfo-title').text.strip()
        
        #  retrieve location 
        location = property.find('span', class_ = 'ListingCell-KeyInfo-address-text').text.strip()
        
        #  retrieving price, bedroom, bathroom, floor area and land size
        #  we will be using exception handling to deal with property listings that lack any or all of these features
        try:  #  for price 
            price = property.find('span', class_ = 'PriceSection-FirstPrice').text.replace('₱','').strip()           
        except AttributeError:
            price = np.nan
        
        try:  #  for num of bedroom
            bedroom_num = property.find('span', class_ = 'icon-bedrooms').next_element.strip()
        except AttributeError:
            bedroom_num = np.nan
        
        try:  #  for num of bathroom
            bathroom_num = property.find('span', class_ = 'icon-bathrooms').next_element.strip()
        except AttributeError:
            bathroom_num = np.nan
        
        try:  #  for floor area
            floor_area = property.find('span', class_ = 'icon-livingsize').next_element.replace('m²','').strip()
        except AttributeError:
            floor_area = np.nan
            
        try:  #  for land size
            land_size  = property.find('span', class_ = 'icon-land_size').next_element.replace('m²','').strip()    
        except AttributeError:
            land_size = np.nan
        
        #  retrievie longitude and latitude
        geopoint = property['data-geo-point'].strip('[\[\]]').split(',')
        longitude = geopoint[0]
        latitude = geopoint[1]
            
        #  appending data to empty lists 
        listing_list.append(listing)
        location_list.append(location)
        price_list.append(price)
        bedroom_num_list.append(bedroom_num)
        bathroom_num_list.append(bathroom_num)
        floor_area_list.append(floor_area)
        land_size_list.append(land_size)
        longitude_list.append(longitude)
        latitude_list.append(latitude)

    #  returning a dataframe
    return dataframe_fromlists(features_list)
    

In [4]:
def dataframe_fromlists(features_list) -> pd.DataFrame:
    '''
    
    This function creates a dataframe from the data lists.

    The function accepts a list of lists containing the data to be converted to a dataframe.
    
    The function returns a dataframe.
    
    '''
    
    df_format = {'listing': features_list[0], 
                'location': features_list[1], 
                'price': features_list[2], 
                'bedroom_num': features_list[3], 
                'bathroom_num': features_list[4], 
                'floor_area': features_list[5],  
                'land_size': features_list[6],
                'longitude': features_list[7],
                'latitude': features_list[8],
                }
    
    #  creating a dataframe from the data lists
    df = pd.DataFrame(data = df_format)
    
    #  returning the dataframe
    return df

In [5]:
def concat_dataframelist(scraped_df_list) -> pd.DataFrame:
    '''
    
    This function concatenates a list of dataframes.

    The function accepts a list of dataframes to be concatenated.
    
    The function returns a dataframe.
    
    '''
    
    #  establishing first dataframe point
    prime_df = scraped_df_list[0]
    
    #  looping through remaining dataframes to concatenate
    for i in range(len(scraped_df_list)-1):
        prime_df = pd.concat([prime_df,scraped_df_list[i+1]], ignore_index=True)
    
    return prime_df 

<h2>Sample Usage of PropertyCrawler</h2>
<p>The following example shows propertyCrawler scraping data for makati properties (sans commercial types) that are for sale and for rent</p>
<p>For each respective category—sale and rent—four different types of properties are scraped:</p>
<ul>
    <li>apartment</li>
    <li>condo</li>
    <li>house</li>
    <li>land</li>
</ul>

In [6]:
makati_condo = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/makati/condominium/buy/?script=23&page=', start=1, end=100)
makati_house = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/makati/house/buy/?script=23&page=', start=1, end=50)
makati_land = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/makati/land/buy/?script=23&page=', start=1, end=21)

Scraping pages:   0%|          | 0/100 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/makati/condominium/buy/?script=23&page=1 --- Success!


Scraping pages:   1%|          | 1/100 [00:01<02:44,  1.67s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/makati/condominium/buy/?script=23&page=2 --- Success!


Scraping pages:   2%|▏         | 2/100 [00:03<02:44,  1.68s/it]

...

Scraping pages:  90%|█████████ | 19/21 [01:20<00:23, 11.62s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/makati/land/buy/?script=23&page=20 --- Success!


Scraping pages:  95%|█████████▌| 20/21 [01:23<00:08,  8.87s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/makati/land/buy/?script=23&page=21 --- Success!


Scraping pages: 100%|██████████| 21/21 [01:25<00:00,  4.08s/it]


In [7]:
rent_makati_apartment = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/makati/apartment/rent/?page=', start=1, end=7)
rent_makati_condo = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/makati/condominium/rent/?page=', start=1, end=100)
rent_makati_house = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/makati/house/rent/?page=', start=1, end=53)
rent_makati_land = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/makati/land/rent/?page=', start=1, end=2)

Scraping pages:   0%|          | 0/7 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/makati/apartment/rent/?page=1 --- Success!


Scraping pages:  14%|█▍        | 1/7 [00:02<00:12,  2.14s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/makati/apartment/rent/?page=2 --- Success!


Scraping pages:  29%|██▊       | 2/7 [00:04<00:10,  2.08s/it]

...

Scraping pages:   0%|          | 0/2 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/makati/land/rent/?page=1 --- Success!


Scraping pages:  50%|█████     | 1/2 [00:06<00:06,  6.82s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/makati/land/rent/?page=2 --- Success!


Scraping pages: 100%|██████████| 2/2 [00:09<00:00,  4.66s/it]


In [8]:
makati_4sale = pd.concat([makati_condo, makati_house, makati_land])
makati_4sale.to_csv('data/for sale/makati_4sale.csv')

makati_4rent = pd.concat([rent_makati_apartment, rent_makati_condo, rent_makati_house, rent_makati_land])
makati_4rent.to_csv('data/for rent/makati_4rent.csv')

In [9]:
taguig_condo = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/taguig/condominium/buy/?script=23&page=', start=1, end=100)
taguig_house = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/taguig/house/buy/?script=23&page=', start=1, end=20)
taguig_land = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/taguig/land/buy/?script=23&page=', start=1, end=15)

Scraping pages:   0%|          | 0/100 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/taguig/condominium/buy/?script=23&page=1 --- Success!


Scraping pages:   1%|          | 1/100 [00:03<05:09,  3.13s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/taguig/condominium/buy/?script=23&page=2 --- Success!


Scraping pages:   2%|▏         | 2/100 [00:05<04:14,  2.60s/it]

...

Scraping pages:  87%|████████▋ | 13/15 [00:32<00:04,  2.30s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/taguig/land/buy/?script=23&page=14 --- Success!


Scraping pages:  93%|█████████▎| 14/15 [00:34<00:02,  2.25s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/taguig/land/buy/?script=23&page=15 --- Success!


Scraping pages: 100%|██████████| 15/15 [00:37<00:00,  2.47s/it]


In [10]:
rent_taguig_apartment = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/taguig/apartment/rent/?page=', start=1, end=2)
rent_taguig_condo = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/taguig/condominium/rent/?page=', start=1, end=100)
rent_taguig_house = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/taguig/house/rent/?page=', start=1, end=8)
rent_taguig_land = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/taguig/land/rent/?page=', start=1, end=2)

Scraping pages:   0%|          | 0/2 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/taguig/apartment/rent/?page=1 --- Success!


Scraping pages:  50%|█████     | 1/2 [00:02<00:02,  2.18s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/taguig/apartment/rent/?page=2 --- Success!


Scraping pages:   0%|          | 0/100 [00:00<?, ?it/s]

...

Scraping pages:   0%|          | 0/2 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/taguig/land/rent/?page=1 --- Success!


Scraping pages:  50%|█████     | 1/2 [00:03<00:03,  3.92s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/taguig/land/rent/?page=2 --- Success!


Scraping pages: 100%|██████████| 2/2 [00:06<00:00,  3.25s/it]


In [11]:
taguig_4sale = pd.concat([taguig_condo, taguig_house, taguig_land])
taguig_4sale.to_csv('data/for sale/taguig_4sale.csv')

taguig_4rent = pd.concat([rent_taguig_apartment, rent_taguig_condo, rent_taguig_house, rent_taguig_land])
taguig_4rent.to_csv('data/for rent/taguig_4rent.csv')

In [12]:
quezon_city_condo = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/quezon-city/condominium/buy/?script=23&page=', start=1, end=100)
quezon_city_house = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/quezon-city/house/buy/?script=23&page=', start=1, end=100)
quezon_city_land = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/quezon-city/land/buy/?script=23&page=', start=1, end=57)

Scraping pages:   0%|          | 0/100 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/quezon-city/condominium/buy/?script=23&page=1 --- Success!


Scraping pages:   1%|          | 1/100 [00:02<04:29,  2.72s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/quezon-city/condominium/buy/?script=23&page=2 --- Success!


Scraping pages:   2%|▏         | 2/100 [00:05<04:34,  2.80s/it]

...

Scraping pages:  96%|█████████▋| 55/57 [02:04<00:04,  2.03s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/quezon-city/land/buy/?script=23&page=56 --- Success!


Scraping pages:  98%|█████████▊| 56/57 [02:06<00:01,  2.00s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/quezon-city/land/buy/?script=23&page=57 --- Success!


Scraping pages: 100%|██████████| 57/57 [02:08<00:00,  2.26s/it]


In [13]:
rent_quezon_city_apartment = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/quezon-city/apartment/rent/?page=', start=1, end=7)
rent_quezon_city_condo = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/quezon-city/condominium/rent/?page=', start=1, end=53)
rent_quezon_city_house = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/quezon-city/house/rent/?page=', start=1, end=12)
rent_quezon_city_land = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/quezon-city/land/rent/?page=', start=1, end=8)

Scraping pages:   0%|          | 0/7 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/quezon-city/apartment/rent/?page=1 --- Success!


Scraping pages:  14%|█▍        | 1/7 [00:04<00:25,  4.21s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/quezon-city/apartment/rent/?page=2 --- Success!


Scraping pages:  29%|██▊       | 2/7 [00:06<00:15,  3.18s/it]

...

Scraping pages:  75%|███████▌  | 6/8 [00:15<00:05,  2.66s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/quezon-city/land/rent/?page=7 --- Success!


Scraping pages:  88%|████████▊ | 7/8 [00:17<00:02,  2.43s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/quezon-city/land/rent/?page=8 --- Success!


Scraping pages: 100%|██████████| 8/8 [00:19<00:00,  2.46s/it]


In [14]:
quezon_city_4sale = pd.concat([quezon_city_condo, quezon_city_house, quezon_city_land])
quezon_city_4sale.to_csv('data/for sale/quezon-city_4sale.csv')

quezon_city_4rent = pd.concat([rent_quezon_city_apartment, rent_quezon_city_condo, rent_quezon_city_house, rent_quezon_city_land])
quezon_city_4rent.to_csv('data/for rent/quezon-city_4rent.csv')

In [15]:
pasig_condo = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/pasig/condominium/buy/?script=23&page=', start=1, end=88)
pasig_house = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/pasig/house/buy/?script=23&page=', start=1, end=63)
pasig_land = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/pasig/land/buy/?script=23&page=', start=1, end=7)

Scraping pages:   0%|          | 0/88 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/pasig/condominium/buy/?script=23&page=1 --- Success!


Scraping pages:   1%|          | 1/88 [00:02<03:46,  2.60s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/pasig/condominium/buy/?script=23&page=2 --- Success!


Scraping pages:   2%|▏         | 2/88 [00:04<03:03,  2.14s/it]

...

Scraping pages:  71%|███████▏  | 5/7 [00:13<00:05,  2.99s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/pasig/land/buy/?script=23&page=6 --- Success!


Scraping pages:  86%|████████▌ | 6/7 [00:15<00:02,  2.75s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/pasig/land/buy/?script=23&page=7 --- Success!


Scraping pages: 100%|██████████| 7/7 [00:18<00:00,  2.61s/it]


In [16]:
rent_pasig_apartment = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/pasig/apartment/rent/?page=', start=1, end=3)
rent_pasig_condo = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/pasig/condominium/rent/?page=', start=1, end=45)
rent_pasig_house = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/pasig/house/rent/?page=', start=1, end=7)
rent_pasig_land = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/pasig/land/rent/?page=', start=1, end=2)

Scraping pages:   0%|          | 0/3 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/pasig/apartment/rent/?page=1 --- Success!


Scraping pages:  33%|███▎      | 1/3 [00:02<00:04,  2.32s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/pasig/apartment/rent/?page=2 --- Success!


Scraping pages:  67%|██████▋   | 2/3 [00:04<00:02,  2.15s/it]

...

Scraping pages:   0%|          | 0/2 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/pasig/land/rent/?page=1 --- Success!


Scraping pages:  50%|█████     | 1/2 [00:01<00:01,  2.00s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/pasig/land/rent/?page=2 --- Success!


Scraping pages: 100%|██████████| 2/2 [00:03<00:00,  1.72s/it]


In [17]:
pasig_4sale = pd.concat([pasig_condo, pasig_house, pasig_land])
pasig_4sale.to_csv('data/for sale/pasig_4sale.csv')

pasig_4rent = pd.concat([rent_pasig_apartment, rent_pasig_condo, rent_pasig_house, rent_pasig_land])
pasig_4rent.to_csv('data/for rent/pasig_4rent.csv')

In [18]:
manila_condo = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/manila/condominium/buy/?script=23&page=', start=1, end=51)
manila_house = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/manila/house/buy/?script=23&page=', start=1, end=24)
manila_land = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/manila/land/buy/?script=23&page=', start=1, end=8)

Scraping pages:   0%|          | 0/51 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/manila/condominium/buy/?script=23&page=1 --- Success!


Scraping pages:   2%|▏         | 1/51 [00:01<01:33,  1.87s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/manila/condominium/buy/?script=23&page=2 --- Success!


Scraping pages:   4%|▍         | 2/51 [00:03<01:31,  1.87s/it]

...

Scraping pages:  75%|███████▌  | 6/8 [00:16<00:05,  2.71s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/manila/land/buy/?script=23&page=7 --- Success!


Scraping pages:  88%|████████▊ | 7/8 [00:19<00:02,  2.80s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/manila/land/buy/?script=23&page=8 --- Success!


Scraping pages: 100%|██████████| 8/8 [00:22<00:00,  2.80s/it]


In [19]:
rent_manila_apartment = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/manila/apartment/rent/?page=', start=1, end=4)
rent_manila_condo = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/manila/condominium/rent/?page=', start=1, end=29)
rent_manila_house = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/manila/house/rent/?page=', start=1, end=1)
rent_manila_land = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/manila/land/rent/?page=', start=1, end=1)

Scraping pages:   0%|          | 0/4 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/manila/apartment/rent/?page=1 --- Success!


Scraping pages:  25%|██▌       | 1/4 [00:02<00:07,  2.41s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/manila/apartment/rent/?page=2 --- Success!


Scraping pages:  50%|█████     | 2/4 [00:04<00:04,  2.17s/it]

...

Scraping pages:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/manila/house/rent/?page=1 --- Success!


Scraping pages:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/manila/land/rent/?page=1 --- Success!


Scraping pages: 100%|██████████| 1/1 [00:02<00:00,  2.54s/it]


In [20]:
manila_4sale = pd.concat([manila_condo, manila_house, manila_land])
manila_4sale.to_csv('data/for sale/manila_4sale.csv')

manila_4rent = pd.concat([rent_manila_apartment, rent_manila_condo, rent_manila_house, rent_manila_land])
manila_4rent.to_csv('data/for rent/manila_4rent.csv')

In [21]:
mandaluyong_condo = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/mandaluyong/condominium/buy/?script=23&page=', start=1, end=69)
mandaluyong_house = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/mandaluyong/house/buy/?script=23&page=', start=1, end=26)
mandaluyong_land = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/mandaluyong/land/buy/?script=23&page=', start=1, end=4)

Scraping pages:   0%|          | 0/69 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/mandaluyong/condominium/buy/?script=23&page=1 --- Success!


Scraping pages:   1%|▏         | 1/69 [00:02<03:18,  2.92s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/mandaluyong/condominium/buy/?script=23&page=2 --- Success!


Scraping pages:   3%|▎         | 2/69 [00:05<02:44,  2.46s/it]

...

Scraping pages:  50%|█████     | 2/4 [00:05<00:04,  2.42s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/mandaluyong/land/buy/?script=23&page=3 --- Success!


Scraping pages:  75%|███████▌  | 3/4 [00:06<00:02,  2.20s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/mandaluyong/land/buy/?script=23&page=4 --- Success!


Scraping pages: 100%|██████████| 4/4 [00:09<00:00,  2.32s/it]


In [22]:
rent_mandaluyong_apartment = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/mandaluyong/apartment/rent/?page=', start=1, end=3)
rent_mandaluyong_condo = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/mandaluyong/condominium/rent/?page=', start=1, end=48)
rent_mandaluyong_house = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/mandaluyong/house/rent/?page=', start=1, end=1)
rent_mandaluyong_land = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/mandaluyong/land/rent/?page=', start=1, end=1)

Scraping pages:   0%|          | 0/3 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/mandaluyong/apartment/rent/?page=1 --- Success!


Scraping pages:  33%|███▎      | 1/3 [00:02<00:04,  2.28s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/mandaluyong/apartment/rent/?page=2 --- Success!


Scraping pages:  67%|██████▋   | 2/3 [00:04<00:01,  1.98s/it]

...

Scraping pages:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/mandaluyong/house/rent/?page=1 --- Success!


Scraping pages:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/mandaluyong/land/rent/?page=1 --- Success!


Scraping pages: 100%|██████████| 1/1 [00:05<00:00,  5.86s/it]


In [23]:
mandaluyong_4sale = pd.concat([mandaluyong_condo, mandaluyong_house, mandaluyong_land])
mandaluyong_4sale.to_csv('data/for sale/mandaluyong_4sale.csv')

mandaluyong_4rent = pd.concat([rent_mandaluyong_apartment, rent_mandaluyong_condo, rent_mandaluyong_house, rent_mandaluyong_land])
mandaluyong_4rent.to_csv('data/for rent/mandaluyong_4rent.csv')

In [24]:
san_juan_condo = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/san-juan-5/condominium/buy/?script=23&page=', start=1, end=22)
san_juan_house = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/san-juan-5/house/buy/?script=23&page=', start=1, end=28)
san_juan_land = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/san-juan-5/land/buy/?script=23&page=', start=1, end=4)

Scraping pages:   0%|          | 0/22 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/san-juan-5/condominium/buy/?script=23&page=1 --- Success!


Scraping pages:   5%|▍         | 1/22 [00:02<00:51,  2.45s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/san-juan-5/condominium/buy/?script=23&page=2 --- Success!


Scraping pages:   9%|▉         | 2/22 [00:04<00:44,  2.23s/it]

...

Scraping pages:  50%|█████     | 2/4 [00:09<00:08,  4.16s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/san-juan-5/land/buy/?script=23&page=3 --- Success!


Scraping pages:  75%|███████▌  | 3/4 [00:12<00:03,  3.74s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/san-juan-5/land/buy/?script=23&page=4 --- Success!


Scraping pages: 100%|██████████| 4/4 [00:15<00:00,  3.84s/it]


In [25]:
rent_san_juan_apartment = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/san-juan-5/apartment/rent/?page=', start=1, end=1)
rent_san_juan_condo = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/san-juan-5/condominium/rent/?page=', start=1, end=6)
rent_san_juan_house = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/san-juan-5/house/rent/?page=', start=1, end=2)
rent_san_juan_land = property_crawler(base_url='https://www.lamudi.com.ph/metro-manila/san-juan-5/land/rent/?page=', start=1, end=1)

Scraping pages:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/san-juan-5/apartment/rent/?page=1 --- Success!


Scraping pages:   0%|          | 0/6 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/san-juan-5/condominium/rent/?page=1 --- Success!


Scraping pages:  17%|█▋        | 1/6 [00:02<00:12,  2.55s/it]

...

Scraping pages:  50%|█████     | 1/2 [00:02<00:02,  2.96s/it]

Retrieving: https://www.lamudi.com.ph/metro-manila/san-juan-5/house/rent/?page=2 --- Success!


Scraping pages:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving: https://www.lamudi.com.ph/metro-manila/san-juan-5/land/rent/?page=1 --- Success!


Scraping pages: 100%|██████████| 1/1 [00:02<00:00,  2.45s/it]


In [26]:
san_juan_4sale = pd.concat([san_juan_condo, san_juan_house, san_juan_land])
san_juan_4sale.to_csv('data/for sale/san_juan_4sale.csv')

san_juan_4rent = pd.concat([rent_san_juan_apartment, rent_san_juan_condo, rent_san_juan_house, rent_san_juan_land])
san_juan_4rent.to_csv('data/for rent/san_juan_4rent.csv')

In [27]:
ncr_4sale = pd.concat([manila_4sale, makati_4sale, mandaluyong_4sale, san_juan_4sale, taguig_4sale, pasig_4sale, quezon_city_4sale])
ncr_4rent = pd.concat([manila_4rent, makati_4rent, mandaluyong_4rent, san_juan_4rent, taguig_4rent, pasig_4rent, quezon_city_4rent])

ncr_4sale.to_csv('data/for sale/ncr_4sale.csv')
ncr_4rent.to_csv('data/for rent/ncr_4rent.csv')

In [28]:
ncr_4sale.shape

(28710, 9)

In [29]:
ncr_4rent.shape

(15270, 9)